In [1]:
from tlspt.datamodules.components.numpy_dataset import NumpyDataset
from tlspt.datamodules.components.base_site import BaseSiteDataset
from tlspt.datamodules.components.octree_dataset import OctreeDataset
from tlspt.datamodules.components.merged_dataset import MergedOctreeDataset

from tlspt.transforms import UniformTLSSampler
from tlspt.models.pointmae.pointmae_seg import PointMAESegmentation
from torch.utils.data import DataLoader
import torch
from matplotlib import pyplot as plt

import numpy as np

In [2]:
seg_dataset = MergedOctreeDataset(
    split_files=['../data/plot_octrees/allen-spain/octrees/____TEST.csv'],
    split='train',
    scales=[2],
    min_points=[512],
    feature_names=['scalar_label'],
    features_to_normalize=None,
    normalize=True,
    transform=UniformTLSSampler(num_points=8192)
    )

seg_dataloader = DataLoader(seg_dataset, batch_size=32, shuffle=True)

2024-11-30 16:04:11.882 | INFO     | tlspt.datamodules.components.base_site:__init__:49 - OctreeDataset(../data/plot_octrees/allen-spain/octrees/____TEST.csv, train, 2): reading splits from ../data/plot_octrees/allen-spain/octrees/____TEST.csv
2024-11-30 16:04:11.885 | INFO     | tlspt.datamodules.components.base_site:__init__:56 - OctreeDataset(../data/plot_octrees/allen-spain/octrees/____TEST.csv, train, 2): looking for 1 folders in ../data/plot_octrees/allen-spain/octrees/
2024-11-30 16:04:11.886 | INFO     | tlspt.datamodules.components.base_site:__init__:64 - OctreeDataset(../data/plot_octrees/allen-spain/octrees/____TEST.csv, train, 2): found 1 plots for 'train' out of 1 plots defined in split file
2024-11-30 16:04:11.887 | INFO     | tlspt.structures.file_octree:__init__:155 - Initializing octree from <class 'str'> ../data/plot_octrees/allen-spain/octrees/SPA01_2m/SPA01.json
2024-11-30 16:04:12.068 | INFO     | tlspt.utils:__init__:40 - Creating directory for stats_file ../data/

In [3]:
seg_dataset.prepare_data()

2024-11-30 16:04:12.233 | INFO     | tlspt.utils:prepare_data:98 - reading stats from ../data/plot_octrees/allen-spain/octrees/stats/stats_23724a02908e6.pkl
2024-11-30 16:04:12.234 | INFO     | tlspt.utils:prepare_data:99 - for dataset OctreeDataset.{'features_to_normalize': None}.train.23724a02908e6
2024-11-30 16:04:12.236 | INFO     | tlspt.utils:prepare_data:108 - mean: tensor([nan])
2024-11-30 16:04:12.237 | INFO     | tlspt.utils:prepare_data:109 - std: tensor([nan])
2024-11-30 16:04:12.237 | INFO     | tlspt.utils:prepare_data:113 - torch.float32
2024-11-30 16:04:12.238 | INFO     | tlspt.utils:prepare_data:117 - Dataset has 1 features named ['scalar_label']. 
 Normalizing None by mean+std. 
2024-11-30 16:04:12.238 | INFO     | tlspt.utils:prepare_data:120 - 3 channels will be zero centered and scaled to [-1,1].


In [4]:
seg_dataset[0]

{'points': tensor([[ 0.1292,  0.1856, -0.0632],
         [-0.0004,  0.2253, -0.0466],
         [-0.1213,  0.2373, -0.0129],
         ...,
         [-0.0384, -0.2776,  0.0064],
         [ 0.1259,  0.1833,  0.0044],
         [ 0.1596,  0.1725,  0.0041]]),
 'features': tensor([[1.],
         [1.],
         [1.],
         ...,
         [0.],
         [1.],
         [1.]]),
 'lengths': 3672,
 'scales': 2}

In [5]:
batch = next(iter(seg_dataloader))

In [6]:
batch['points'].shape
batch['features'].shape
batch['lengths'].shape
batch['scales'].shape

torch.Size([32])

In [7]:
model = PointMAESegmentation(neighbor_alg='ball_query', ball_radius=0.2, scale=2.0)

In [8]:
fix_batch = {'points': batch['points'], 'features': torch.nan_to_num(batch['features']), 'lengths': batch['lengths'], 'scales': batch['scales']}

In [9]:
model(fix_batch)

tensor([[[ 0.1966,  0.4764],
         [ 0.4629,  0.0971],
         [-0.1560, -0.4572],
         ...,
         [ 1.0219, -0.0927],
         [-0.3144,  0.2421],
         [ 0.4767,  0.7248]],

        [[ 0.2230, -0.2814],
         [ 0.8494,  0.1385],
         [ 1.4164, -0.0927],
         ...,
         [ 0.1885, -0.1892],
         [ 0.4503, -0.3875],
         [ 0.1306, -0.5259]],

        [[ 0.1303,  0.0369],
         [ 0.1299,  0.1771],
         [-0.2318, -0.1314],
         ...,
         [-0.3535,  0.0608],
         [ 0.2351, -0.0972],
         [-0.0607, -0.0787]],

        ...,

        [[ 0.1525,  0.3273],
         [ 0.1690,  0.4984],
         [-0.2036, -0.0173],
         ...,
         [ 0.3992, -0.3065],
         [ 0.3578,  0.0131],
         [ 0.3481,  0.3062]],

        [[ 0.0714,  0.0826],
         [-0.2872,  0.2767],
         [ 0.2399,  0.5633],
         ...,
         [ 0.5541,  0.7157],
         [-0.1244,  0.2121],
         [-0.3609,  0.2578]],

        [[ 0.5370,  0.7942],
       

In [10]:
B, N, _ = batch['points'].shape
print(B, N)

32 8192


In [11]:
x_gt = batch['features']
print(x_gt.shape)
x_gt = x_gt.squeeze(-1)
print(x_gt.shape)

torch.Size([32, 8192, 1])
torch.Size([32, 8192])


In [12]:
patches, centers = model.group(
    batch['points'], batch['lengths']
)
print(patches.shape)
print(centers.shape)

torch.Size([32, 64, 32, 3])
torch.Size([32, 64, 3])


In [13]:
patch_embeddings = model.patch_encoder(patches)
print(patch_embeddings.shape)
pos_embeddings = model.pos_encoder(centers)
print(pos_embeddings.shape)

torch.Size([32, 64, 384])
torch.Size([32, 64, 384])


In [14]:
x, feature_list = model.transformer_encoder(patch_embeddings, pos_embeddings, feature_blocks=model.feature_blocks)
print(model.feature_blocks)
print("===")
print(x.shape)
print([f.shape for f in feature_list])

[3, 7, 11]
===
torch.Size([32, 64, 384])
[torch.Size([32, 64, 384]), torch.Size([32, 64, 384]), torch.Size([32, 64, 384])]


In [15]:
feature_tensor = torch.cat(feature_list, dim=2)
feature_tensor = feature_tensor.transpose(1, 2)
print(feature_tensor.shape)

torch.Size([32, 1152, 64])


In [16]:
x_max = torch.max(feature_tensor, dim=2, keepdim=True)[0]
x_avg = torch.mean(feature_tensor, dim=2, keepdim=True)
print(x_max.shape)
print(x_avg.shape)

torch.Size([32, 1152, 1])
torch.Size([32, 1152, 1])


In [17]:
x_max_feature = x_max.expand(-1, -1, N)
x_avg_feature = x_avg.expand(-1, -1, N)
print(x_max_feature.shape)
print(x_avg_feature.shape)

torch.Size([32, 1152, 8192])
torch.Size([32, 1152, 8192])


In [18]:
x_global_feature = torch.cat([x_max_feature, x_avg_feature], dim=1)
print(x_global_feature.shape)

torch.Size([32, 2304, 8192])


In [19]:
f_level_0 = model.propagation_0( #Note - this seems to suck up most of the computer.
    batch['points'].transpose(-1, -2), centers.transpose(-1, -2), batch['points'].transpose(-1, -2), feature_tensor
)

In [20]:
f_level_0.shape

torch.Size([32, 1024, 8192])

In [21]:
x = torch.cat(
    (f_level_0, x_global_feature), dim=1
)
print(x.shape)

torch.Size([32, 3328, 8192])


In [22]:
x = model.relu(model.bns1(model.convs1(x)))
x = model.dp1(x)
print(x.shape)
x = model.relu(model.bns2(model.convs2(x)))
print(x.shape)
x = model.convs3(x)
print(x.shape)

torch.Size([32, 512, 8192])
torch.Size([32, 256, 8192])
torch.Size([32, 2, 8192])


In [23]:
x_hat = x.transpose(1, 2)
print(x_hat.shape)

torch.Size([32, 8192, 2])


In [24]:
loss = model.get_loss(x_hat, x_gt.long( ))

In [25]:
print(loss)

tensor(0.7326, grad_fn=<NllLossBackward0>)


In [26]:
x_pred = torch.argmax(x_hat, dim=2).long()

In [27]:
model.get_acc(x_pred, x_gt.long())

tensor(0.4779)

In [28]:
model.get_miou(x_pred, x_gt.long())

tensor(0.2642)

In [29]:
x_gt.shape

torch.Size([32, 8192])

In [30]:
x_pred.shape

torch.Size([32, 8192])